In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import NearestNeighbors

/var/folders/34/gby6tkr103xfh0fy_017m5grcjxgrv/T/ipykernel_31883/2481694947.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
data_inmet = pd.read_csv('../../model/additional_data/datasets/inmet_2019_2024.csv')
data_modis = pd.read_csv('../../model/data/modisTodos.csv')

In [4]:
# Verificar os tipos de dados das colunas de latitude e longitude
print("Tipos de dados em data_inmet:")
print(data_inmet.dtypes)
print("\nTipos de dados em data_modis:")
print(data_modis.dtypes)

Tipos de dados em data_inmet:
Data                                                      object
latitude                                                  object
longitude                                                 object
PRECIPITACAO TOTAL, HORARIO (mm)                         float64
PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)    float64
RADIACAO GLOBAL (KJ/m2)                                  float64
TEMPERATURA DO AR - BULBO SECO, HORARIA (0C)             float64
UMIDADE RELATIVA DO AR, HORARIA (%)                      float64
VENTO, RAJADA MAXIMA (m/s)                               float64
dtype: object

Tipos de dados em data_modis:
latitude      float64
longitude     float64
confidence    float64
dtype: object


In [5]:
# Verificar se há valores nulos
print("\nValores nulos em data_inmet:")
print(data_inmet[['latitude', 'longitude']].isnull().sum())
print("\nValores nulos em data_modis:")
print(data_modis[['latitude', 'longitude']].isnull().sum())


Valores nulos em data_inmet:
latitude     0
longitude    0
dtype: int64

Valores nulos em data_modis:
latitude     0
longitude    0
dtype: int64


In [6]:
# Converter as colunas para string, se necessário
data_inmet['latitude'] = data_inmet['latitude'].astype(str).str.replace(',', '.')
data_inmet['longitude'] = data_inmet['longitude'].astype(str).str.replace(',', '.')

data_modis['latitude'] = data_modis['latitude'].astype(str).str.replace(',', '.')
data_modis['longitude'] = data_modis['longitude'].astype(str).str.replace(',', '.')

In [7]:
# Ajustar os tipos de dados para latitude e longitude
data_inmet['latitude'] = data_inmet['latitude'].astype(float)
data_inmet['longitude'] = data_inmet['longitude'].astype(float)
data_modis['latitude'] = data_modis['latitude'].astype(float)
data_modis['longitude'] = data_modis['longitude'].astype(float)

In [8]:
# Arredondar coordenadas para combinar dados com tolerância
data_inmet['latitude'] = data_inmet['latitude'].round(4)
data_inmet['longitude'] = data_inmet['longitude'].round(4)
data_modis['latitude'] = data_modis['latitude'].round(4)
data_modis['longitude'] = data_modis['longitude'].round(4)

In [9]:
# Preparar os dados para Nearest Neighbors
inmet_coords = data_inmet[['latitude', 'longitude']].values
modis_coords = data_modis[['latitude', 'longitude']].values
nbrs = NearestNeighbors(n_neighbors=1, radius=0.01).fit(modis_coords)
distances, indices = nbrs.kneighbors(inmet_coords)

In [10]:
# Adicionar índice de correspondência e distância ao DataFrame do INMET
data_inmet['modis_index'] = indices.flatten()
data_inmet['distance'] = distances.flatten()

In [11]:
# Filtrar correspondências dentro da tolerância
tolerance = 0.1
filtered_inmet = data_inmet[data_inmet['distance'] < tolerance]

In [49]:
# Fazer o merge
combined_data_tolerance = pd.merge(filtered_inmet, data_modis, left_on='modis_index', right_index=True, how='inner')

# Verifique o DataFrame combinado com tolerância
print("DataFrame Combinado com Tolerância:")
print(combined_data_tolerance.head())

DataFrame Combinado com Tolerância:
         Data  latitude_x  longitude_x  PRECIPITACAO TOTAL, HORARIO (mm)  \
0  2019-01-01    -15.7893     -47.9258                               1.4   
1  2019-01-02    -15.7893     -47.9258                               0.0   
2  2019-01-03    -15.7893     -47.9258                               0.0   
3  2019-01-04    -15.7893     -47.9258                               0.0   
4  2019-01-05    -15.7893     -47.9258                               1.0   

   PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)  \
0                                             888.02       
1                                             888.80       
2                                             887.62       
3                                             887.54       
4                                             887.96       

   RADIACAO GLOBAL (KJ/m2)  TEMPERATURA DO AR - BULBO SECO, HORARIA (0C)  \
0                  1192.79                                         20.

In [50]:
print(combined_data_tolerance.isna().sum())  # Para verificar quantos NaNs existem em cada coluna

Data                                                     0
latitude_x                                               0
longitude_x                                              0
PRECIPITACAO TOTAL, HORARIO (mm)                         0
PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)    0
RADIACAO GLOBAL (KJ/m2)                                  0
TEMPERATURA DO AR - BULBO SECO, HORARIA (0C)             0
UMIDADE RELATIVA DO AR, HORARIA (%)                      0
VENTO, RAJADA MAXIMA (m/s)                               0
modis_index                                              0
distance                                                 0
latitude_y                                               0
longitude_y                                              0
confidence                                               0
dtype: int64


In [51]:
# Se o DataFrame combinado estiver vazio
if combined_data_tolerance.empty:
    print("O DataFrame combinado está vazio. Tentando um merge 'outer' para verificar divergências...")
    combined_data_tolerance = pd.merge(data_inmet, data_modis, on=['latitude', 'longitude'], how='outer')
    print(combined_data_tolerance.head())  # Exibir as primeiras linhas para verificar divergências
# 5. Modelagem (continua a partir daqui, se o DataFrame não estiver vazio)
if not combined_data_tolerance.empty:
    features = combined_data_tolerance[['PRECIPITACAO TOTAL, HORARIO (mm)', 'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)',
                              'RADIACAO GLOBAL (KJ/m2)', 'TEMPERATURA DO AR - BULBO SECO, HORARIA (0C)',
                              'UMIDADE RELATIVA DO AR, HORARIA (%)', 'VENTO, RAJADA MAXIMA (m/s)']]
    target = combined_data_tolerance['confidence']  # Certifique-se de que 'confidence' existe

In [52]:
print(combined_data_tolerance.head()) 

         Data  latitude_x  longitude_x  PRECIPITACAO TOTAL, HORARIO (mm)  \
0  2019-01-01    -15.7893     -47.9258                               1.4   
1  2019-01-02    -15.7893     -47.9258                               0.0   
2  2019-01-03    -15.7893     -47.9258                               0.0   
3  2019-01-04    -15.7893     -47.9258                               0.0   
4  2019-01-05    -15.7893     -47.9258                               1.0   

   PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)  \
0                                             888.02       
1                                             888.80       
2                                             887.62       
3                                             887.54       
4                                             887.96       

   RADIACAO GLOBAL (KJ/m2)  TEMPERATURA DO AR - BULBO SECO, HORARIA (0C)  \
0                  1192.79                                         20.52   
1                  2300.98    

In [13]:
# Função para calcular o confidence com base nas condições
def calcular_confidence(dados_linha):
    contagem_verdadeiras = 0
    if dados_linha['PRECIPITACAO TOTAL, HORARIO (mm)'] < 600:
        contagem_verdadeiras += 1
    if dados_linha['PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)'] < 1000:
        contagem_verdadeiras += 1
    if dados_linha['RADIACAO GLOBAL (KJ/m2)'] > 1200:
        contagem_verdadeiras += 1
    if dados_linha['TEMPERATURA DO AR - BULBO SECO, HORARIA (0C)'] > 30:
        contagem_verdadeiras += 1
    if dados_linha['UMIDADE RELATIVA DO AR, HORARIA (%)'] > 40:
        contagem_verdadeiras += 1
    if dados_linha['VENTO, RAJADA MAXIMA (m/s)'] > 5:
        contagem_verdadeiras += 1

    if contagem_verdadeiras == 6:
        return 100
    elif contagem_verdadeiras == 5 or contagem_verdadeiras == 4:
        return 75
    elif contagem_verdadeiras == 3:
        return 50
    elif contagem_verdadeiras == 2 or contagem_verdadeiras == 1:
        return 25
    else:
        return 0

In [53]:
# Dividir os dados em treinamento e teste
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [54]:
# Criar e treinar o modelo de árvore de decisão
model = DecisionTreeRegressor(random_state=42)
model.fit(x_train, y_train)

DecisionTreeRegressor(random_state=42)

In [58]:
# Fazer previsões e avaliar o modelo
predictions = model.predict(x_test)

In [71]:
print(predictions)

[84.  0. 37. ... 77. 37. 78.]


In [60]:
# Verifique os nomes das colunas do DataFrame combinado
print(combined_data_tolerance.columns)

Index(['Data', 'latitude_x', 'longitude_x', 'PRECIPITACAO TOTAL, HORARIO (mm)',
       'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)',
       'RADIACAO GLOBAL (KJ/m2)',
       'TEMPERATURA DO AR - BULBO SECO, HORARIA (0C)',
       'UMIDADE RELATIVA DO AR, HORARIA (%)', 'VENTO, RAJADA MAXIMA (m/s)',
       'modis_index', 'distance', 'latitude_y', 'longitude_y', 'confidence'],
      dtype='object')


In [72]:
# Criar o DataFrame com as previsões de confidence
predictions_df = pd.DataFrame(predictions, columns=['predicted_confidence'])

In [73]:
# Selecionar as colunas latitude_x, longitude_x, e confidence de combined_data_tolerance
dados_existentes = combined_data_tolerance[['latitude_x', 'longitude_x', 'confidence']].reset_index(drop=True)

In [74]:
# Concatenar dados existentes e as novas previsões (predictions_df)
previsao_df = pd.concat([dados_existentes, predictions_df], axis=1)

In [75]:
# Verificar previsao_df para garantir que as colunas foram combinadas corretamente
print(previsao_df.head())

   latitude_x  longitude_x  confidence  predicted_confidence
0    -15.7893     -47.9258        84.0                  84.0
1    -15.7893     -47.9258        84.0                   0.0
2    -15.7893     -47.9258        84.0                  37.0
3    -15.7893     -47.9258        84.0                  78.0
4    -15.7893     -47.9258        84.0                   0.0


In [26]:
# Salvar os resultados em um arquivo CSV
previsao_df.to_csv('resultados_previsao_decisionTree.csv', index=False)